In [1]:
!pip install -U huggingface_hub
!pip install -U datasets
!pip install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

In [2]:
import pandas as pd

from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Path selection
# file_path_gl = '/content/drive/My Drive/4º Carrera/NLP/Proyecto/gl_dataset.csv'  # PATH: Luis
file_path_gl = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/gl_dataset.csv'  # PATH: Iker

# file_path_pt = '/content/drive/My Drive/4º Carrera/NLP/Proyecto/pt_dataset.csv'  # PATH: Luis
file_path_pt = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/pt_dataset.csv'  # PATH: Iker

In [5]:
# Cargar ambos datasets
df_pt = pd.read_csv(file_path_pt)
df_gl = pd.read_csv(file_path_gl)

# Seleccionar subconjuntos de cada dataset
def select_subset(df, train_size, test_size, val_size):
    train = df[df['split'] == 'train'].head(train_size)
    test = df[df['split'] == 'test'].head(test_size)
    val = df[df['split'] == 'val'].head(val_size)
    return pd.concat([train, test, val])

# Seleccionar 5 de train, 2 de test, y 2 de val para cada idioma
df_pt_subset = select_subset(df_pt, train_size=5, test_size=2, val_size=2)
df_gl_subset = select_subset(df_gl, train_size=5, test_size=2, val_size=2)

# Verificar resultados
print(f"Portugués: {len(df_pt_subset)} filas seleccionadas")
print(f"Gallego: {len(df_gl_subset)} filas seleccionadas")

Portugués: 9 filas seleccionadas
Gallego: 9 filas seleccionadas


In [6]:
df_subset_combined = pd.concat([df_pt_subset, df_gl_subset])
print(f"Total de filas en el conjunto combinado: {len(df_subset_combined)}")

Total de filas en el conjunto combinado: 18


In [7]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
for idx, row in df_subset_combined.iterrows():
    input_text = "summarize: " + row['text']
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids)

    print(tokenizer.decode(outputs[0]))

<pad>summary: Deve a Ré ser condenada a pagar ao Autor
<pad>summary: O valor equivalente a 9/12 de uma remuneraç
<pad>summary: O Acórdio recorrido está, apes
<pad>summary: Há, por isso, erro de julgamen
<pad>summary: Ora é pacificamente aceite que na t
<pad>summary: As agricultores, o Arguido B foi inscrit
<pad>summary: Efectivamente, o arguido B nunca desenvolve
<pad> summary: - Por força das publicaç<unk>es no Correio da
<pad>- O acompanho e sofrimento da filha
<pad> summary: Pequeno homenaxe a mellor pel<unk>cula de
<pad> <unk> ltimo nome: Wilhelm Bunsen (1811-99). Qu<unk>mic
<pad>summary: O Parlamento receita bicicletas para combater a contaminación
<pad> summary: As asociacións estudiant<unk>s apoian a folg
<pad>summary: O venres visitáronnos os de Son a A


Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors


<pad> NON <unk> ORDENANZA C<unk> VICA. Senior. Prevedi<unk>
<pad> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
<pad>Unha paréntese neste blog. Vou a colgar un post que
<pad>A conselleira de Sanidade, Pilar Farjas, anunciou este mé
